# Stats and Analysis Notebook

In [95]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import ttest_1samp
import sqlite3
import re
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

read clean data

In [122]:
gross_budget=pd.read_csv('../Data/clean_data/gross_budget.csv',index_col=0)

In [124]:
movie_genre_profit=pd.read_csv('../Data/clean_data/gross_budget_genre.csv',index_col=0)

In [125]:
gross_budget.head()

,title,foreign_gross,production_budget,domestic_gross_y,worldwide_gross,net_profit,ROI
0,Toy Story 3,652000000,200000000,415004880,1068879522,868879522,434.439761
1,Inception,535700000,160000000,292576195,835524642,675524642,422.202901
2,Shrek Forever After,513900000,165000000,238736787,756244673,591244673,358.330105
3,The Twilight Saga: Eclipse,398000000,68000000,300531751,706102828,638102828,938.386512
4,Iron Man 2,311500000,170000000,312433331,621156389,451156389,265.386111


In [126]:
movie_genre_profit.head()

,title,foreign_gross,production_budget,domestic_gross_y,worldwide_gross,net_profit,ROI,runtime_minutes,genres,averagerating,numvotes
0,Toy Story 3,652000000,200000000,415004880,1068879522,868879522,434.439761,103.0,"Adventure,Animation,Comedy",8.3,682218.0
1,Inception,535700000,160000000,292576195,835524642,675524642,422.202901,148.0,"Action,Adventure,Sci-Fi",8.8,1841066.0
2,Shrek Forever After,513900000,165000000,238736787,756244673,591244673,358.330105,93.0,"Adventure,Animation,Comedy",6.3,167532.0
3,The Twilight Saga: Eclipse,398000000,68000000,300531751,706102828,638102828,938.386512,124.0,"Adventure,Drama,Fantasy",5.0,211733.0
4,Iron Man 2,311500000,170000000,312433331,621156389,451156389,265.386111,124.0,"Action,Adventure,Sci-Fi",7.0,657690.0


Analysis and recommendation

In [129]:
movie_genre_profit=movie_genre_profit.assign(genres=movie_genre_profit.genres.str.split(",")).explode('genres')

In [130]:
movie_genre_profit.head()

,title,foreign_gross,production_budget,domestic_gross_y,worldwide_gross,net_profit,ROI,runtime_minutes,genres,averagerating,numvotes
0,Toy Story 3,652000000,200000000,415004880,1068879522,868879522,434.439761,103.0,Adventure,8.3,682218.0
0,Toy Story 3,652000000,200000000,415004880,1068879522,868879522,434.439761,103.0,Animation,8.3,682218.0
0,Toy Story 3,652000000,200000000,415004880,1068879522,868879522,434.439761,103.0,Comedy,8.3,682218.0
1,Inception,535700000,160000000,292576195,835524642,675524642,422.202901,148.0,Action,8.8,1841066.0
1,Inception,535700000,160000000,292576195,835524642,675524642,422.202901,148.0,Adventure,8.8,1841066.0
